In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mpl_toolkits

%matplotlib inline

In [2]:
train = pd.read_csv('clear_data.csv')
data = train.sample(frac=0.7, random_state=5)
test = train.loc[~train.index.isin(data.index)]

In [3]:
def detect_outliers_zscore(copy, label):
    thres = 4
    data = copy[label]
    mean = np.mean(data)
    std = np.std(data)
    for i in range(0, len(data)):
        if i in data.index:
            element = data[i]
            z_score = (element-mean)/std
            if (np.abs(z_score) > thres):
                copy.at[i, label] = mean
#detect_outliers_zscore(data, 'latitude')
#detect_outliers_zscore(data, 'rooms')

In [4]:
'''def detect_outliers_zscore(data, label):
    outliers = []
    thres = 4
    data = data[label]
    mean = np.mean(data)
    std = np.std(data)
    for i in range(0, len(data)):
        element = data[i]
        z_score = (element-mean)/std
        if (np.abs(z_score) > thres):
            outliers.append(i)
    return outliers# Driver code
#sample_outliers = detect_outliers_zscore(data, 'longitude')
sample_outliers = detect_outliers_zscore(data, 'latitude')
sample_outliers += detect_outliers_zscore(data, 'rooms')
data = data.drop(sample_outliers)'''

"def detect_outliers_zscore(data, label):\n    outliers = []\n    thres = 4\n    data = data[label]\n    mean = np.mean(data)\n    std = np.std(data)\n    for i in range(0, len(data)):\n        element = data[i]\n        z_score = (element-mean)/std\n        if (np.abs(z_score) > thres):\n            outliers.append(i)\n    return outliers# Driver code\n#sample_outliers = detect_outliers_zscore(data, 'longitude')\nsample_outliers = detect_outliers_zscore(data, 'latitude')\nsample_outliers += detect_outliers_zscore(data, 'rooms')\ndata = data.drop(sample_outliers)"

In [5]:
to_drop = []
for key in data.keys():
    if data[key].dtype == 'int64':
        true = 0
        false = 0
        for element in data[key]:
            if element == 1:
                true += 1
            elif element == 0:
                false += 1
        if true + false == len(data[key]):
            if true * 250 < false:
                #print(true, false, true * 25)
                to_drop.append(key)
print(len(to_drop))

columns = data.columns.tolist()
columns = [c for c in columns if c not in to_drop]
data = data[columns]

66


In [6]:
#Import libraries:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV
import matplotlib.pylab as plt
import mpl_toolkits
# import pandas for data wrangling
import pandas as pd


# import numpy for Scientific computations
import numpy as np


# import machine learning libraries
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

xgbr = xgb.XGBRegressor(seed=0)
target = 'price'
columns = data.columns.tolist()
columns = [c for c in columns if c not in ['price']]
xgbr.fit(data[columns], data[target])

predictions = xgbr.predict(test[columns])

lin_mse = mean_squared_error(predictions, test[target], squared=False)
print("Computed error:", lin_mse)

Computed error: 69897.17636071787


In [13]:
xgbr = xgb.XGBRegressor(seed=0)
train = pd.read_csv('clear_data.csv')
data = train.sample(frac=0.7, random_state=5)
columns = data.columns.tolist()
xgbr.fit(data[columns], data[target])
predictions = xgbr.predict(test[columns])

lin_mse = mean_squared_error(predictions, test[target], squared=False)
print("Computed error:", lin_mse)

Computed error: 5404.660151030323


In [8]:
#data.to_csv('engineered_test.csv', index=False)